In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import yfinance as yf
import pickle
import os
import math
from ReferentialData.polygonData import PolygonAPI
from DataAnalysis import DataExtensions
from Filtering.KalmanFilter import KalmanFilter

In [2]:
import BacktestEngine.backtest as backtest
import BacktestEngine.order_book as order_book

In [ ]:
## End Date
end_dt = "2025-01-01"
## Start date
start_dt = "2020-01-01"
# Loading "pairs trade" buckets
DIR = r'C:\Users\raymo\OneDrive\Desktop\Ray Stuff\_Cache'
COR_DIR = r'\Users\raymo\OneDrive\Desktop\Playground\Financial-Modelling-Playground\Quant_Trading\Clustering'
# with open(os.path.join(COR_DIR, "correlation_buckets_no_shorts.pkl"), "rb") as file:
#     BUCKETS = pickle.load(file)

# # Concatenating dictionary value lists
# UNIVERSE=[]
# for i in list(BUCKETS.values()):
#     UNIVERSE.extend(i)

In [5]:
UNIVERSE = ['QQQ', 'SPX', 'AAAU', 'ACWI'] 

In [7]:
colnames = ["Close", "Volume"]
DataLoader = PolygonAPI()
MID_LOADED = False
HIGH_LOADED = False
if not MID_LOADED:
    mid_df = {}
if not HIGH_LOADED:
    high_df = {}
### loading in low frequency data
if len(mid_df) == 0:
    mid_df = DataLoader.getPrices(UNIVERSE, timespan= "day", from_ = start_dt, 
                                    to_=end_dt,
                                  logDir=DIR, _parallel=True)
    MID_LOADED = True

### loading in high frequency data
if len(high_df) == 0:
    high_df = DataLoader.getPrices(UNIVERSE,from_ = start_dt, 
                                    to_=end_dt, timespan="minute", logDir=DIR, _parallel=True, override=False)
    HIGH_LOADED = True


In [ ]:
### add random trades


In [ ]:
BBO = order_book.Book()
HOLDING_PERIOD_CHECK = {}
isLong = False
hasPosition = False

def isFirstTrade(ticker, date):
    if ticker not in HOLDING_PERIOD_CHECK:
        return True
    if date not in HOLDING_PERIOD_CHECK[ticker]:
        return True
    return False

pairsLEGS = [x for x in LEGS if x != reference_ticker]

for _, row in df.iloc[window:].iterrows():
    if (row.BuySell == 1 
        and not hasPosition
        and isFirstTrade(reference_ticker, row.Date.normalize())):
        print('bought spread at ', row.Date)
        BBO.addOrder(row.Date, reference_ticker, row[reference_ticker],10 * row[f'{reference_ticker}_hr'])
        for LEG in pairsLEGS:
            BBO.addOrder(row.Date, LEG, row[LEG], -10 * row[f'{LEG}_hr'])
        if reference_ticker not in HOLDING_PERIOD_CHECK:
            HOLDING_PERIOD_CHECK[reference_ticker] = []
        HOLDING_PERIOD_CHECK[reference_ticker].append(row.Date.normalize())
        isLong = True
        hasPosition = True
    elif (row.BuySell == -1 
        and not hasPosition
        and isFirstTrade(reference_ticker, row.Date.normalize())):
        print('sold spread at ', row.Date)
        BBO.addOrder(row.Date, reference_ticker, row[reference_ticker], -10 * row[f'{reference_ticker}_hr'])
        for LEG in pairsLEGS:
            BBO.addOrder(row.Date, LEG, row[LEG], 10 * row[f'{LEG}_hr'])
        isLong = False
        if reference_ticker not in HOLDING_PERIOD_CHECK:
            HOLDING_PERIOD_CHECK[reference_ticker] = []
        HOLDING_PERIOD_CHECK[reference_ticker].append(row.Date.normalize())
        hasPosition = True
        isLong = False
    else:
        if hasPosition and isFirstTrade(reference_ticker, row.Date.normalize()):
            print('exit spread at ', row.Date)
            prices = {}
            for ticker in LEGS:
                prices[ticker] = row[ticker]
            BBO.unwindALL(row.Date, prices =prices)
            if reference_ticker not in HOLDING_PERIOD_CHECK:
                HOLDING_PERIOD_CHECK[reference_ticker] = []
                HOLDING_PERIOD_CHECK[reference_ticker].append(row.Date.normalize())
            hasPosition = False
            isLong = False

#BBO.addOrder(df_res.iloc[100]["Date"], "TSM", 300.0, 10)
#BBO.addOrder(df_res.iloc[120]["Date"], "TSM", 250.0, 25)
#BBO.addOrder(df_res.iloc[200]["Date"], "TSM", 350.0, -30)
#BBO.getTickerBook(["TSLA"]).tail(1)#.loc['Quantity']
#BBO.get_history()

#BBO.addOrder(df_res.iloc[50]["Date"], "TSMC34.SA", 300.0, 10)
#BBO.addOrder(df_res.iloc[150]["Date"], "TSMC34.SA", 250.0, -25)